# Client-Side Code

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model

from PIL import Image

### Inference Model

In [15]:
# Half of the Inference Model Stored on Client Side
class ClientResNet(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super(ClientResNet, self).__init__(*args, **kwargs)
        
        # Load ResNet50 pre-trained model without top (fully connected) layers
        resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        resnet_base.trainable = False
        
        # Get the output of the first three convolutional layers
        middle_layer = resnet_base.get_layer('conv3_block4_out')
        self.seq0 = Model(inputs=resnet_base.input, outputs=middle_layer.output)

    # Convert image into right dimensions
    def preprocess_image(self, img):
        img_array = kimage.img_to_array(img)
        expand_img = np.expand_dims(img_array, axis=0)
        return preprocess_input(expand_img)

    # Forward pass of the client model
    def predict(self, image_path):
        img = Image.open(image_path)
        img = img.resize((224, 224)).convert("RGB")

        preprocessed_img = self.preprocess_image(img)
        return self.seq0(preprocessed_img)

In [7]:
# Load in Model
ClientModel = ClientResNet()

# Perform Inference on image path
image_path = "jeans.png"
client_embeddings = ClientModel.predict(image_path)

# Replace: This is a mimic for sending to server
np.save("client_embeddings.npy", client_embeddings)

In [16]:
# Add Laplacian Noise
client_embeddings = ClientModel.predict(image_path)
noise_mean, noise_scale = 0.05, 0.1
laplace_noise = np.random.laplace(noise_mean, noise_scale, size=client_embeddings.shape)
client_embeddings += laplace_noise
np.save("laplace_embeddings.npy", client_embeddings)

In [9]:
# Add Siamese Noise